In [ ]:
import pandas as pd
import gradio as gr
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import LabelEncoder

# === BACA DATASET ===
df = pd.read_excel("Dataset SistemPakar.xlsx")

X = df.drop("Kerusakan", axis=1)
y = df["Kerusakan"]

# === ENCODING LABEL ===
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# === MODEL ===
model = BernoulliNB()
model.fit(X, y_encoded)

# === SOLUSI UNTUK SETIAP KERUSAKAN ===
solusi_dict = {
    'Oli Mesin Habis': '🛢️ Tambahkan atau ganti oli mesin secara berkala.',
    'Seal Klep Bocor': '🔧 Ganti seal klep agar oli tidak bocor ke ruang bakar.',
    'Masalah pada Busi': '🔋 Periksa dan ganti busi jika sudah aus.',
    'Filter Udara Kotor': '🌬️ Bersihkan atau ganti filter udara.',
    'Masalah pada CVT': '⚙️ Periksa dan ganti v-belt atau roller CVT jika perlu.',
    'CVT Aus': '🔧 Ganti komponen CVT yang aus.',
    'Aki Lemah': '🔋 Periksa tegangan aki, ganti jika lemah.',
    'Ring Seher Aus': '🔩 Ganti ring seher yang aus.',
    'Kerusakan Roller CVT': '⚙️ Ganti roller CVT.'
}

# === FUNGSI DIAGNOSA ===
def diagnosa(*args):
    user_input = list(args)
    if all(val == 0 for val in user_input):
        return """
        <div style="border: 2px solid #ccc; border-radius: 10px; padding: 15px; background-color: #fffbe6;">
            <h3 style="color: #ff9800; text-align: center;">⚠️ Tidak Ada Gejala yang Dipilih</h3>
            <p>Silakan pilih minimal satu gejala untuk melakukan diagnosa.</p>
        </div>
        """
    prediksi = model.predict([user_input])
    hasil = le.inverse_transform(prediksi)[0]
    solusi = solusi_dict.get(hasil, "Solusi tidak ditemukan. Silakan konsultasi ke bengkel.")
    proba = model.predict_proba([user_input])[0]
    proba_list = "".join([
        f"<li><b>{le.inverse_transform([i])[0]}</b>: {p*100:.2f}%</li>"
        for i, p in enumerate(proba)
    ])

    return f"""
    <div style="border: 2px solid #4CAF50; border-radius: 10px; padding: 15px; background-color: #f0fff0;">
        <h3 style="color: #4CAF50; text-align: center;">✅ <u>Hasil Diagnosa</u></h3>
        <p><b>Kerusakan Teridentifikasi:</b> <span style="color: red; font-size: 1.2em;">{hasil}</span></p>
        <p><b>Solusi Disarankan:</b> {solusi}</p>
        <details>
            <summary><b>📊 Lihat Detail Probabilitas</b></summary>
            <ul>{proba_list}</ul>
        </details>
    </div>
    """

# === FUNGSI TAMPILKAN SEMUA SOLUSI ===
def semua_solusi():
    solusi_html = "".join([
        f"<li><b>{kerusakan}</b>: {solusi_dict[kerusakan]}</li>"
        for kerusakan in sorted(solusi_dict.keys())
    ])
    return f"""
    <div style="border: 2px dashed #2196F3; border-radius: 10px; padding: 15px; background-color: #e3f2fd;">
        <h3 style="color: #2196F3;">💡 Daftar Solusi Semua Kerusakan</h3>
        <ul>{solusi_html}</ul>
    </div>
    """

# === ANTARMUKA GRADIO ===
with gr.Blocks(theme=gr.themes.Base()) as demo:
    gr.Markdown("""
    <div style='text-align: center; font-size: 2em; font-weight: bold; color: #4CAF50;'>
    🚀 SISTEM PAKAR DIAGNOSA MOTOR MATIC 🚀
    </div>
    <div style='text-align: center; font-size: 1.2em; color: #555;'>
    Pilih gejala motor Anda:
    </div>
    """)

    checkbox_inputs = []
    with gr.Row():
        col1 = gr.Column()
        col2 = gr.Column()
        for i, gejala in enumerate(X.columns):
            cb = gr.Checkbox(label=gejala)
            checkbox_inputs.append(cb)

    tombol_diagnosa = gr.Button("🔍 Diagnosa Sekarang", variant="primary")
    tombol_solusi = gr.Button("💡 Lihat Solusi Semua Kerusakan", variant="secondary")
    hasil_output = gr.HTML()

    tombol_diagnosa.click(fn=diagnosa, inputs=checkbox_inputs, outputs=hasil_output)
    tombol_solusi.click(fn=semua_solusi, inputs=[], outputs=hasil_output)

    gr.Markdown("<hr><center>© 2024 Sistem Pakar Motor Matic </center>")

demo.launch(share=True)
